In [12]:
import pandas as pd
import sqlite3
import pymysql.cursors
from dotenv import load_dotenv
import requests
import zipfile
import shutil
import math

# Load environment variables from the .env file
load_dotenv()
import os

In [19]:
url = "https://static.nhtsa.gov/nhtsa/downloads/CRSS/2016/CRSS2016CSV.zip"

response = requests.get(url).content
filename = "CRSS2016CSV"
zip_file = filename + ".zip"
with open(zip_file, "wb") as f:
        f.write(response)

# Unzip the file
with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(filename)

In [14]:
csv_path = '%s/vehicle.csv' % filename
if not os.path.isfile(csv_path):
    print(csv_path)
    print('here')

CRSS2021CSV/vehicle.csv
here


In [15]:
shutil.rmtree(filename)
os.remove(zip_file)

In [9]:
csv_path = 'CRSS2021CSV/CRSS2021CSV/vpicdecode.csv'
chunk_generator = pd.read_csv(csv_path, encoding="ISO-8859-1", chunksize=1000)

first = next(chunk_generator)
for index, row in first.iloc[:].iterrows():
    if index == 0:
        cols = "`,`".join([str(i) for i in first.columns.tolist()])
        print(cols)
    elif index == 1:
        print(row['VehicleType'])

# Continue iterating through the rest of the chunks
#for chunk in chunk_generator:
#    for index, row in chunk.iterrows():
#        # Process each row in subsequent chunks
#        if index < 10:
#            print(row.keys())
#        else:
#            continue

CASENUM`,`VEH_NO`,`VehicleDescriptor`,`VINDecodedOn`,`VINDecodeError`,`VehicleTypeId`,`VehicleType`,`ManufacturerFullNameId`,`ManufacturerFullName`,`MakeId`,`Make`,`ModelId`,`Model`,`ModelYear`,`Series`,`Trim`,`Series2`,`Trim2`,`PlantCountryId`,`PlantCountry`,`PlantState`,`PlantCity`,`PlantCompanyName`,`DestinationMarketId`,`DestinationMarket`,`BasePrice`,`Note`,`BodyClassId`,`BodyClass`,`DoorsCount`,`Windows`,`WheelBaseTypeId`,`WheelBaseType`,`TrackWidthIN`,`GrossVehicleWeightRatingFromId`,`GrossVehicleWeightRatingFrom`,`GrossVehicleWeightRatingToId`,`GrossVehicleWeightRatingTo`,`GrossCombWeightRatingFromId`,`GrossCombWeightRatingFrom`,`GrossCombWeightRatingToId`,`GrossCombWeightRatingTo`,`CurbWeightLB`,`WheelBaseIN_from`,`WheelBaseIN_To`,`WheelsCount`,`WheelSizeFrontIN`,`WheelSizeRearIN`,`TruckBodyCabTypeId`,`TruckBodyCabType`,`TruckBedTypeId`,`TruckBedType`,`TruckBedLengthIN`,`BusTypeId`,`BusType`,`BusFloorConfigurationTypeId`,`BusFloorConfigurationType`,`BusLengthFT`,`OtherBusInfo`

In [16]:
# Connect to the database
connection = pymysql.connect(host=os.getenv("DATABASE_HOST"),
                             user=os.getenv("DATABASE_USERNAME"),
                             password=os.getenv("DATABASE_PASSWORD"),
                             database=os.getenv("DATABASE"),
                             ssl={'ca': os.getenv("CA_CERT_PATH")},
                             cursorclass=pymysql.cursors.DictCursor)

date_range = range(2016, 2022)
# for d in date_range:
#     url = "https://static.nhtsa.gov/nhtsa/downloads/CRSS/%s/CRSS%sCSV.zip" % (d, d)
#     response = requests.get(url)
#     filename = "CRSS%sCSV" % d
#     zip_file = filename + ".zip"
#     with open(zip_file, "wb") as f:
#         f.write(response.content)

#     # Unzip the file
#     with zipfile.ZipFile(zip_file, "r") as zip_ref:
#         zip_ref.extractall(filename)

with connection:
    with connection.cursor() as cursor:
        # sqlQuery = "CREATE TABLE Accidents(Case_Num varchar(32), VEH_Num int, Occupation_Num int, Hit_Run int, VIN varchar(32), Model_Year varchar(32), Make int, Model int)"
        for d in date_range:
            url = "https://static.nhtsa.gov/nhtsa/downloads/CRSS/%s/CRSS%sCSV.zip" % (d, d)
            response = requests.get(url)
            filename = "CRSS%sCSV" % d
            zip_file = filename + ".zip"
            with open(zip_file, "wb") as f:
                f.write(response.content)

            # Unzip the file
            with zipfile.ZipFile(zip_file, "r") as zip_ref:
                zip_ref.extractall(filename)
            csv_path = '%s/vehicle.csv' % filename
            if not os.path.isfile(csv_path):
                csv_path = '%s/VEHICLE.CSV' % filename
                if not os.path.isfile(csv_path):
                    csv_path = '%s/VEHICLE.csv' % filename
                    if not os.path.isfile(csv_path):
                        csv_path = '%s/%s/vehicle.csv' % (filename, filename)
            chunk_generator = pd.read_csv(csv_path, chunksize=1000, encoding="ISO-8859-1", usecols=['CASENUM', 'VEH_NO', 'NUMOCCS', 'HIT_RUN', 'VIN', 'MDLYR_IM', 'MAKE', 'MODEL'])

            # first = next(chunk_generator)
            for chunk in chunk_generator:
                batch = []
                for index, row in chunk.iterrows():
                    id = str(row['CASENUM']) + '-' + str(row['VEH_NO'])
                    batch.append((id, row['CASENUM'], row['VEH_NO'], row['NUMOCCS'], row['HIT_RUN'], row['VIN'], row['MDLYR_IM'], row['MAKE'], row['MODEL']))
                cursor.executemany('INSERT INTO Accidents (id, Case_Num, VEH_Num, Occupation_Num, Hit_Run, VIN, Model_Year, Model, Make) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)', batch)
                        # if index == 0:
                        #     print(row.keys())
                        # elif index == 1:
                        #     print(row['VIN'])
                # Execute "SHOW TABLES" query
                # cursor.execute(sqlQuery)
            connection.commit()


                # cursor.execute("SHOW TABLES")

                # Fetch all the rows
                # tables = cursor.fetchall()

                # Print out the tables
                # print("Tables in the database:")
                # for table in tables:
                #     print(table)
                # Delete the zip file
            shutil.rmtree(filename)
            os.remove(zip_file)


In [16]:
# Connect to the database
connection = pymysql.connect(host=os.getenv("DATABASE_HOST"),
                             user=os.getenv("DATABASE_USERNAME"),
                             password=os.getenv("DATABASE_PASSWORD"),
                             database=os.getenv("DATABASE"),
                             ssl={'ca': os.getenv("CA_CERT_PATH")},
                             cursorclass=pymysql.cursors.DictCursor)

date_range = range(2016, 2022)
# for d in date_range:
#     url = "https://static.nhtsa.gov/nhtsa/downloads/CRSS/%s/CRSS%sCSV.zip" % (d, d)
#     response = requests.get(url)
#     filename = "CRSS%sCSV" % d
#     zip_file = filename + ".zip"
#     with open(zip_file, "wb") as f:
#         f.write(response.content)

#     # Unzip the file
#     with zipfile.ZipFile(zip_file, "r") as zip_ref:
#         zip_ref.extractall(filename)
cols = ['CASENUM', 'VEH_NO', 'VehicleDescriptor', 'VehicleTypeId', 'VehicleType', 'ManufacturerFullNameId', 'ManufacturerFullName', 'MakeId', 'Make', 'ModelId', 'Model', 'PlantCountryId', 'PlantCountry', 'PlantState', 'PlantCity', 'PlantCompanyName', 'BasePrice']
with connection:
    with connection.cursor() as cursor:
        # sqlQuery = "CREATE TABLE Accidents(Case_Num varchar(32), VEH_Num int, Occupation_Num int, Hit_Run int, VIN varchar(32), Model_Year varchar(32), Make int, Model int)"
        for d in date_range:
            url = "https://static.nhtsa.gov/nhtsa/downloads/CRSS/%s/CRSS%sCSV.zip" % (d, d)
            response = requests.get(url)
            filename = "CRSS%sCSV" % d
            zip_file = filename + ".zip"
            with open(zip_file, "wb") as f:
                f.write(response.content)

            # Unzip the file
            with zipfile.ZipFile(zip_file, "r") as zip_ref:
                zip_ref.extractall(filename)
            csv_path = '%s/vpicdecode.csv' % filename
            if not os.path.isfile(csv_path):
                csv_path = '%s/VPICDECODE.CSV' % filename
                if not os.path.isfile(csv_path):
                    csv_path = '%s/VPICDECODE.csv' % filename
                    if not os.path.isfile(csv_path):
                        csv_path = '%s/%s/vpicdecode.csv' % (filename, filename)
            chunk_generator = pd.read_csv(csv_path, chunksize=1000, encoding="ISO-8859-1", usecols=cols)

            # first = next(chunk_generator)
            for chunk in chunk_generator:
                chunk.fillna(-1, inplace=True)
                batch = []
                for index, row in chunk.iterrows():
                    batch.append((row['CASENUM'], row['VEH_NO'], row['VehicleDescriptor'], row['VehicleTypeId'], row['VehicleType'], row['ManufacturerFullNameId'], row['ManufacturerFullName'], row['MakeId'], row['Make'], row['ModelId'], row['Model'], row['PlantCountryId'], row['PlantCountry'], row['PlantState'], row['PlantCity'], row['PlantCompanyName'], row['BasePrice']))
                q = 'INSERT INTO vpicdecode (CASENUM, VEH_NO, VehicleDescriptor, VehicleTypeId, VehicleType, ManufacturerFullNameId, ManufacturerFullName, MakeId, Make, ModelId, Model, PlantCountryId, PlantCountry, PlantState, PlantCity, PlantCompanyName, BasePrice) values (' + '%s,' * (len(cols) - 1) + '%s)'
                cursor.executemany(q, batch)
                        # if index == 0:
                        #     print(row.keys())
                        # elif index == 1:
                        #     print(row['VIN'])
                # Execute "SHOW TABLES" query
                # cursor.execute(sqlQuery)
            connection.commit()


                # cursor.execute("SHOW TABLES")

                # Fetch all the rows
                # tables = cursor.fetchall()

                # Print out the tables
                # print("Tables in the database:")
                # for table in tables:
                #     print(table)
                # Delete the zip file
            shutil.rmtree(filename)
            os.remove(zip_file)


TypeError: fillna() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [ ]:
sqlQuery = "CREATE TABLE Accidents(Case_Num varchar(32), VEH_Num int, Occupation_Num int, Hit_Run int, VIN varchar(32), Model_Year varchar(32), Make int, Model int)"

sqlQuery2 = "CREATE TABLE vpicdecode(CASENUM varchar(31),VEH_NO int,VehicleDescriptor varchar(255),VehicleTypeId int,VehicleType varchar(255),ManufacturerFullNameId int,ManufacturerFullName varchar(255),MakeId int,Make varchar(127),ModelId int,Model varchar(127),PlantCountryId int,PlantCountry varchar(63),PlantState varchar(63),PlantCity varchar(63),PlantCompanyName varchar(63),BasePrice dec, PRIMARY KEY (CASENUM, VEH_NO));"

In [51]:
def add_to_db(row):
    # create a cursor
    c = conn.cursor()

    cols = ", ".join(row.keys())

    sql = '''
        INSERT INTO accidents_table (''' + cols + '''
        ) VALUES (
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
        )
    '''
    all_values = []
    for (key, value) in row.items():
        all_values.append(value)
    # Execute the SQL command with the provided data
    r = tuple(all_values)
    print(r)
    c.execute(sql, r)


chunk_generator = pd.read_csv("US_Accidents_March23.csv", chunksize=1000)
conn = sqlite3.connect('accidents.sqlite')

# Continue iterating through the rest of the chunks
for chunk in chunk_generator:
    for index, row in chunk.iterrows():
        # Process each row in subsequent chunks
        add_to_db(row)


# commit the changes
conn.commit()
# close the connection
conn.close()

('A-1', 'Source2', 3, '2016-02-08 05:46:00', '2016-02-08 11:00:00', 39.865147, -84.058723, nan, nan, 0.01, 'Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.', 'I-70 E', 'Dayton', 'Montgomery', 'OH', '45424', 'US', 'US/Eastern', 'KFFO', '2016-02-08 05:58:00', 36.9, nan, 91.0, 29.68, 10.0, 'Calm', nan, 0.02, 'Light Rain', False, False, False, False, False, False, False, False, False, False, False, False, False, 'Night', 'Night', 'Night', 'Night')


OperationalError: near "(": syntax error